In [1]:
import os
import glob
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.preprocessing import image

from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions


In [2]:
train_data_path = os.path.join('..', 'Dataset', 'data', 'train') # '/content/data/train'
valid_data_path = os.path.join('..', 'Dataset', 'data', 'test') # '/content/data/test'

train_data_agumentation = ImageDataGenerator(rescale = 1./255,
                                            #  shear_range = 0.2,
                                             zoom_range = 0.1
                                            #  horizontal_flip = True,
                                            #  vertical_flip = True
                                             )

val_data_agumentation = ImageDataGenerator(rescale = 1./255)


# load training data
train_data = train_data_agumentation.flow_from_directory(directory = train_data_path,
                                                         target_size = (224,224),
                                                         class_mode = 'categorical',
                                                         batch_size = 64)

val_data = val_data_agumentation.flow_from_directory(directory = valid_data_path,
                                                     target_size = (224,224),
                                                     class_mode = 'categorical',
                                                     batch_size = 64)

Found 3216 images belonging to 3 classes.
Found 1584 images belonging to 3 classes.


In [3]:
checkpoint_dir = 'checkpoints_ResNet50_adam'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_filepath = os.path.join(checkpoint_dir,
                                   "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}_val_loss_{val_loss:.2f}.keras")

checkpoint = ModelCheckpoint(filepath = checkpoint_filepath,
                             monitor = 'val_loss',
                             verbose = 0,
                             save_best_only = True,
                             save_weights_only = False,
                             mode = 'auto')

early = EarlyStopping(monitor = 'val_loss',
                      min_delta = 0,
                      patience = 10,
                      verbose = 0,
                      mode = 'auto')

reduceLR = ReduceLROnPlateau(monitor="val_loss", 
                             factor=0.1,
                             patience=10, 
                             verbose=0, 
                             mode="auto",
                             min_delta=0.0001, 
                             cooldown=0,
                             min_lr=0.0)

csv_logger = CSVLogger('training.log')

callbacks_list = [checkpoint, reduceLR, csv_logger] #, early


In [4]:
image_size = [224, 224]
num_classes = len(glob.glob(train_data_path+'/*'))

# Load the pre-trained model
model = ResNet50(input_shape = image_size + [3], 
                 weights = 'imagenet', 
                 include_top = False)

# our layers - you can add more if you want
x = Flatten()(model.output)

output_layer = Dense(num_classes, activation='softmax')(x)

# model.layers.trainable = False
for layer in model.layers:
    layer.trainable = False

# create a model object
model = Model(inputs = model.input, outputs = output_layer)
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

                                                                                                  
 conv2_block2_add (Add)      (None, 56, 56, 256)          0         ['conv2_block1_out[0][0]',    
                                                                     'conv2_block2_3_bn[0][0]']   
                                                                                                  
 conv2_block2_out (Activati  (None, 56, 56, 256)          0         ['conv2_block2_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 56, 56, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_blo

 D)                                                                                               
                                                                                                  
 conv3_block2_2_bn (BatchNo  (None, 28, 28, 128)          512       ['conv3_block2_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_2_relu (Activ  (None, 28, 28, 128)          0         ['conv3_block2_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 28, 28, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
          

 conv4_block1_1_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block1_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_1_relu (Activ  (None, 14, 14, 256)          0         ['conv4_block1_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_2_conv (Conv2  (None, 14, 14, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalizati

                                                                                                  
 conv4_block3_3_bn (BatchNo  (None, 14, 14, 1024)         4096      ['conv4_block3_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block3_add (Add)      (None, 14, 14, 1024)         0         ['conv4_block2_out[0][0]',    
                                                                     'conv4_block3_3_bn[0][0]']   
                                                                                                  
 conv4_block3_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_blo

 rmalization)                                                                                     
                                                                                                  
 conv4_block6_2_relu (Activ  (None, 14, 14, 256)          0         ['conv4_block6_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block6_3_conv (Conv2  (None, 14, 14, 1024)         263168    ['conv4_block6_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 14, 14, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
          

 conv5_block3_1_conv (Conv2  (None, 7, 7, 512)            1049088   ['conv5_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv5_block3_1_bn (BatchNo  (None, 7, 7, 512)            2048      ['conv5_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_1_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 7, 7, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)       

In [6]:
#Initializing the hyperparameters
adam = Adam(learning_rate = 0.001, 
            beta_1 = 0.9,
            beta_2 = 0.999,
            epsilon = 1e-07)

# compile the model with adam optimizer, categorical_croosentropy loss function
model.compile(optimizer = adam,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])


In [ ]:
history = model.fit(train_data,
                    batch_size = 64,
                    epochs = 100,
                    validation_data = val_data,
                    callbacks = callbacks_list)

# steps_per_epoch=len(training_set),
# validation_steps=len(test_set),

model.save('ResNet50_adam_model_last_epoch.h5')

Epoch 1/100


51/51 [==============================] - 479s 9s/step - loss: 1.5450 - accuracy: 0.4502 - val_loss: 1.8869 - val_accuracy: 0.3504 - lr: 0.0010
Epoch 2/100
51/51 [==============================] - 441s 9s/step - loss: 1.2119 - accuracy: 0.4540 - val_loss: 1.0635 - val_accuracy: 0.4836 - lr: 0.0010
Epoch 3/100
51/51 [==============================] - 535s 11s/step - loss: 1.0590 - accuracy: 0.4953 - val_loss: 1.3158 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 4/100
51/51 [==============================] - 529s 10s/step - loss: 1.2490 - accuracy: 0.4695 - val_loss: 1.1391 - val_accuracy: 0.4874 - lr: 0.0010
Epoch 5/100
51/51 [==============================] - 639s 13s/step - loss: 1.1615 - accuracy: 0.4888 - val_loss: 0.9608 - val_accuracy: 0.5436 - lr: 0.0010
Epoch 6/100
51/51 [==============================] - 694s 14s/step - loss: 0.9900 - accuracy: 0.5227 - val_loss: 1.0277 - val_accuracy: 0.5032 - lr: 0.0010
Epoch 7/100
51/51 [==============================] - 635s 13s/st

51/51 [==============================] - 415s 8s/step - loss: 0.7749 - accuracy: 0.6542 - val_loss: 1.0367 - val_accuracy: 0.5221 - lr: 1.0000e-06
Epoch 50/100
51/51 [==============================] - 418s 8s/step - loss: 0.7743 - accuracy: 0.6536 - val_loss: 1.0360 - val_accuracy: 0.5208 - lr: 1.0000e-06
Epoch 51/100
51/51 [==============================] - 418s 8s/step - loss: 0.7745 - accuracy: 0.6486 - val_loss: 1.0359 - val_accuracy: 0.5215 - lr: 1.0000e-06
Epoch 52/100
51/51 [==============================] - 438s 9s/step - loss: 0.7776 - accuracy: 0.6511 - val_loss: 1.0328 - val_accuracy: 0.5215 - lr: 1.0000e-06
Epoch 53/100
51/51 [==============================] - 454s 9s/step - loss: 0.7782 - accuracy: 0.6511 - val_loss: 1.0333 - val_accuracy: 0.5215 - lr: 1.0000e-07
Epoch 54/100
51/51 [==============================] - 452s 9s/step - loss: 0.7796 - accuracy: 0.6433 - val_loss: 1.0338 - val_accuracy: 0.5215 - lr: 1.0000e-07
Epoch 55/100
51/51 [==============================] -

In [ ]:
# accuracies
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.savefig('ResNet50_adam_model_accuracy_and_val_accuracy.png', dpi=200)
plt.show()
 
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('ResNet50_adam_model_loss_and_val_loss.png', dpi=200)
plt.show()


In [ ]:
score = model.evaluate(train_data, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

In [ ]:
score = model.evaluate(val_data, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])